In [ ]:
import os
import re
from pykospacing import Spacing
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
# import ace_tools as tools
import ace_tools_open as tools
import pickle
import seaborn as sns
import matplotlib.pyplot as plt


from konlpy.tag import Okt
okt = Okt()

In [ ]:
# 파일 읽기
with open('data.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# 공백 제거 (모든 공백 문자 삭제)
content = "".join(content.split())

# 문장 분리 (정규 표현식 사용)
sentences = re.split(r'[.!?]', content)

# 빈 문장 제거 및 큰따옴표 제거
clean_sentences = [re.sub(r'"', '', sentence) for sentence in sentences if sentence]

# 파일에 저장
with open('no_space_sentence.txt', 'w', encoding='utf-8') as output_file:
    output_file.write("\n".join(clean_sentences))



In [ ]:
# 성능 좋음 -> 이걸로 띄여쓰기 복원을 하고 있다.

# pykospacing 객체 생성
spacing = Spacing()

# 입력 파일 읽기
with open('no_space_sentence.txt', 'r', encoding='utf-8') as input_file:
    sentences = input_file.readlines()

# 공백 복원
spaced_sentences = [spacing(sentence.strip()) for sentence in sentences]

# 결과 저장
with open('yes_space_sentence.txt', 'w', encoding='utf-8') as output_file:
    output_file.write("\n".join(spaced_sentences))


In [ ]:
# 1️⃣ Sentence-BERT 모델 로드 (한국어 지원 모델 사용) -> 로그인 하고 귀찮아서 그냥 다른거 쓴다
model = SentenceTransformer('jhgan/ko-sbert-sts')


# 2️⃣ 파일에서 문장 읽기
with open('yes_space_sentence.txt', 'r', encoding='utf-8') as file:
    sentences = [line.strip() for line in file.readlines() if line.strip()]

# 3️⃣ 문장 임베딩 변환 (각 문장을 벡터로 변환)
sentence_embeddings = model.encode(sentences)

# 4️⃣ 문장 간 유사도 계산 (코사인 유사도 활용)
similarity_matrix = util.pytorch_cos_sim(sentence_embeddings, sentence_embeddings).numpy()

# 5️⃣ 데이터프레임 생성 (문장 간 유사도 표시)
df_similarity = pd.DataFrame(similarity_matrix, index=sentences, columns=sentences)

# 파일로 저장
df_similarity.to_excel("sentence_similarity.xlsx", index=True)

In [ ]:
# 1️⃣ Sentence-BERT 모델 로드 (한국어 지원 모델 사용) -> 로그인 하고 귀찮아서 그냥 다른거 쓴다
model = SentenceTransformer('jhgan/ko-sbert-sts')


# 2️⃣ 파일에서 문장 읽기
with open('yes_space_sentence.txt', 'r', encoding='utf-8') as file:
    sentences = [line.strip() for line in file.readlines() if line.strip()]

# 3️⃣ 문장 임베딩 변환 (각 문장을 벡터로 변환)
sentence_embeddings = model.encode(sentences)

# 4️⃣ 문장 간 유사도 계산 (코사인 유사도 활용)
similarity_matrix = util.pytorch_cos_sim(sentence_embeddings, sentence_embeddings).numpy()

# 5️⃣ 데이터프레임 생성 (문장 간 유사도 표시)
df_similarity = pd.DataFrame(similarity_matrix, index=sentences, columns=sentences)

# 파일로 저장
with open("sentence_similarity.pkl", "wb") as pkl_file:
    pickle.dump(df_similarity, pkl_file)

In [ ]:
# 그래프로 표현

# 1️⃣ 저장된 pickle 파일 로드
with open("sentence_similarity.pkl", "rb") as pkl_file:
    df_similarity = pickle.load(pkl_file)

# 2️⃣ 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(df_similarity, annot=False, cmap="coolwarm", xticklabels=False, yticklabels=False)

# 3️⃣ 그래프 타이틀 추가
plt.title("Sentence Similarity Heatmap", fontsize=14)
plt.xlabel("Sentences", fontsize=12)
plt.ylabel("Sentences", fontsize=12)

# 4️⃣ 시각화 출력
plt.show()


In [ ]:
# 1️⃣ 저장된 pickle 파일 로드
with open("sentence_similarity.pkl", "rb") as pkl_file:
    df_similarity = pickle.load(pkl_file)

# 2️⃣ 유사도 기준 설정 (0.8 ≤ 유사도 < 0.9)
threshold_low = 0.8
threshold_high = 0.9  # 0.9 이상은 제외

# 3️⃣ 유사도가 지정된 범위에 속하는 문장 쌍 추출 (자기 자신 제외)
similar_sentences = []
for i in range(len(df_similarity)):
    for j in range(i + 1, len(df_similarity)):  # 대각선 기준 중복 제거
        similarity_score = round(df_similarity.iloc[i, j], 6)  # 소수점 6자리까지 반올림
        if threshold_low <= similarity_score < threshold_high:  # 0.8 이상, 0.9 미만
            sentence1 = df_similarity.index[i]
            sentence2 = df_similarity.columns[j]
            similar_sentences.append((sentence1, sentence2, similarity_score))

# 4️⃣ 결과를 DataFrame으로 변환
df_high_similarity = pd.DataFrame(similar_sentences, columns=["문장1", "문장2", "유사도"])

# 5️⃣ 유사도 높은 순으로 정렬
df_high_similarity = df_high_similarity.sort_values(by="유사도", ascending=False)

# 6️⃣ 결과 출력 (상위 10개)
print(df_high_similarity.head(10))  # 더 보고 싶으면 숫자 조정 가능

# 7️⃣ 파일로 저장 (옵션)
df_high_similarity.to_excel("high_similarity_08_09.xlsx", index=False)




# 하하하하하ㅏ하 " 이거를 지워야하는구나